<a href="https://colab.research.google.com/github/Andyzxm/GEOG_520_open_source_method/blob/main/A21_Xinming_Zhang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import cohen_kappa_score as kap
from sklearn.metrics import accuracy_score as oa
from sklearn.metrics import classification_report as cp
from sklearn.metrics import roc_auc_score as rocauc
from sklearn.metrics import precision_recall_curve as pr 
from sklearn.metrics import auc
from sklearn.preprocessing import MinMaxScaler

Task 1. The “HeartDisease” column is the dependent variable. It is currently coded such
that 0 = No Heart Disease and 1 = Heart Disease. Make sure this variable is treated as a
nominal variable in the model. The following predictor variables should be treated as
continuous: “Age”, “RestingBP”, “Cholesterol”, “MaxHR”, and “Oldpeak”. The
following predictor variable should be treated as nominal data: “Sex”,
“ChestPainType”, “RestingECG”, “FastingBS”, “ExerciseAngina”, and “ST_Slope”.
Make sure all variables are treated appropriately in the model

In [ ]:
#Task1 
heartd=pd.read_csv("/content/drive/MyDrive/GEOG520_python_data/datasets/datasets/heart.csv")
heartd.columns
heartd.columns= [i.replace(" ", "_") for i in heartd.columns]
heartd.columns

Index(['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
       'HeartDisease'],
      dtype='object')

In [ ]:
heartd=heartd[["HeartDisease","Sex","ChestPainType","RestingECG","FastingBS","ExerciseAngina","ST_Slope", "Age","RestingBP","Cholesterol","MaxHR","Oldpeak"]]
heartd.dropna(axis=0)
heartd

,HeartDisease,Sex,ChestPainType,RestingECG,FastingBS,ExerciseAngina,ST_Slope,Age,RestingBP,Cholesterol,MaxHR,Oldpeak
0,0,M,ATA,Normal,0,N,Up,40,140,289,172,0.0
1,1,F,NAP,Normal,0,N,Flat,49,160,180,156,1.0
2,0,M,ATA,ST,0,N,Up,37,130,283,98,0.0
3,1,F,ASY,Normal,0,Y,Flat,48,138,214,108,1.5
4,0,M,NAP,Normal,0,N,Up,54,150,195,122,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,1,M,TA,Normal,0,N,Flat,45,110,264,132,1.2
914,1,M,ASY,Normal,1,N,Flat,68,144,193,141,3.4
915,1,M,ASY,Normal,0,Y,Flat,57,130,131,115,1.2
916,1,F,ATA,LVH,0,N,Flat,57,130,236,174,0.0


Task 2. Subset the variables to create models using (1) all the available predictor
variables, (2) just “Cholesterol” and “MaxHR”, and (3) “Age”, “RestingBP”, and
“MaxHR”.

In [ ]:
#task2 
M1=heartd[["Sex","ChestPainType","RestingECG","FastingBS","ExerciseAngina","ST_Slope","Age","RestingBP","Cholesterol","MaxHR","Oldpeak"]]
M2= heartd[["Cholesterol","MaxHR"]]
M3= heartd[["Age","RestingBP","MaxHR"]]

In [ ]:
M1=M1.dropna(axis=0)
M1

,Sex,ChestPainType,RestingECG,FastingBS,ExerciseAngina,ST_Slope,Age,RestingBP,Cholesterol,MaxHR,Oldpeak
0,M,ATA,Normal,0,N,Up,40,140,289,172,0.0
1,F,NAP,Normal,0,N,Flat,49,160,180,156,1.0
2,M,ATA,ST,0,N,Up,37,130,283,98,0.0
3,F,ASY,Normal,0,Y,Flat,48,138,214,108,1.5
4,M,NAP,Normal,0,N,Up,54,150,195,122,0.0
...,...,...,...,...,...,...,...,...,...,...,...
913,M,TA,Normal,0,N,Flat,45,110,264,132,1.2
914,M,ASY,Normal,1,N,Flat,68,144,193,141,3.4
915,M,ASY,Normal,0,Y,Flat,57,130,131,115,1.2
916,F,ATA,LVH,0,N,Flat,57,130,236,174,0.0


In [ ]:

M2

,Cholesterol,MaxHR
0,289,172
1,180,156
2,283,98
3,214,108
4,195,122
...,...,...
913,264,132
914,193,141
915,131,115
916,236,174


In [ ]:
M3

,Age,RestingBP,MaxHR
0,40,140,172
1,49,160,156
2,37,130,98
3,48,138,108
4,54,150,122
...,...,...,...
913,45,110,132
914,68,144,141
915,57,130,115
916,57,130,174


Task 3. Split the data into separate and non-overlapping training and testing sets using
the “HeartDisease” variable for stratification.

In [ ]:
#Task3 split data for training using "HeartDisease" variable
#add adummies for nominal variables 
#Model 1
Sex= pd.get_dummies(M1.Sex, drop_first= True)
ChestPainType=pd.get_dummies(M1.ChestPainType,drop_first= True)
RestingECG=pd.get_dummies(M1.RestingECG,drop_first= True)
FastingBS=pd.get_dummies(M1.FastingBS,drop_first= True)
ExerciseAngina=pd.get_dummies(M1.ExerciseAngina,drop_first= True)
ST_Slope=pd.get_dummies(M1.ST_Slope,drop_first= True)
y=pd.concat([heartd,Sex,RestingECG,FastingBS,ExerciseAngina,ST_Slope], axis=1)
x1= y.drop(columns=["Sex","ChestPainType","RestingECG","FastingBS","ExerciseAngina","ST_Slope"])
y1=y["HeartDisease"].to_numpy()


In [ ]:
x_train, x_test,y_train, y_test = train_test_split(x1,y1,test_size= 0.11, stratify= y1, random_state= 42)

In [ ]:
#m2
x2=M2
y2=heartd[["HeartDisease"]].to_numpy()
x_train2, x_test2,y_train2, y_test2 = train_test_split(x2,y2,test_size= 0.5, stratify= y2, random_state= 42)
#m3
x3=M3
y3=heartd[["HeartDisease"]].to_numpy()
x_train3, x_test3,y_train3, y_test3 = train_test_split(x3,y3,test_size= 0.33, stratify= y3, random_state= 42)

Task 4. Train three Random Forest models using the three different feature spaces. You
can use the default hyperparameter settings. 

In [ ]:
#Task 4. 
#rf1
rf1= RandomForestClassifier(n_estimators= 100, max_features=11)
rf1= rf1.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
#rf2
rf2= RandomForestClassifier(n_estimators= 100, max_features=2)
rf2=rf2.fit(x_train2,y_train2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#rf3
rf3= RandomForestClassifier(n_estimators= 100, max_features=3)
rf3=rf3.fit(x_train3,y_train3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


Task 5. Assess the models using the withheld testing data. Calculate Confusion Matrices
and Overall Accuracy metrics. Using the precited probabilities, calculate the Area
Under the Receiver Operating Characteristic (ROC) Curve (AUC ROC). Use the results
to discuss and compare the different feature spaces

In [ ]:
#model 1 with all variables 
pred= rf1.predict(x_test)
print(cm(y_test,pred))
print(oa(y_test,pred))
print(kap(y_test,pred))
print(cp(y_test,pred))

[[45  0]
 [ 0 56]]
1.0
1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        45
           1       1.00      1.00      1.00        56

    accuracy                           1.00       101
   macro avg       1.00      1.00      1.00       101
weighted avg       1.00      1.00      1.00       101



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
#model 2 with 2 variables 
pred= rf2.predict(x_test2)
print(cm(y_test2,pred))
print(oa(y_test2,pred))
print(kap(y_test2,pred))
print(cp(y_test2,pred))

[[127  78]
 [ 71 183]]
0.6753812636165577
0.34110812451226913
              precision    recall  f1-score   support

           0       0.64      0.62      0.63       205
           1       0.70      0.72      0.71       254

    accuracy                           0.68       459
   macro avg       0.67      0.67      0.67       459
weighted avg       0.67      0.68      0.67       459



In [ ]:
#model 3 with 3variables 
pred= rf3.predict(x_test3)
print(cm(y_test3,pred))
print(oa(y_test3,pred))
print(kap(y_test3,pred))
print(cp(y_test3,pred))

[[ 87  48]
 [ 52 116]]
0.66996699669967
0.3339488261672382
              precision    recall  f1-score   support

           0       0.63      0.64      0.64       135
           1       0.71      0.69      0.70       168

    accuracy                           0.67       303
   macro avg       0.67      0.67      0.67       303
weighted avg       0.67      0.67      0.67       303



In [ ]:
#ROC/AUC ROC
rfc_prob1= rf1.predict_proba(x_test)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
#M1
y_TEST=y_test
y_TEST[y_TEST ==" No Heart Disease"]=0
y_TEST[y_TEST =="Heart Disease"] = 1
print(rocauc(y_TEST, rfc_prob1[:,1]))



In [ ]:
#M2
rfc_prob2= rf2.predict_proba(x_test2)
y_TEST2=y_test2
y_TEST2[y_TEST2 ==" No Heart Disease"]=0
y_TEST2[y_TEST2 =="Heart Disease"] = 1
print(rocauc(y_TEST2, rfc_prob2[:,1]))

0.7301325139235644


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """


In [ ]:
#M3
rfc_prob3= rf3.predict_proba(x_test3)
y_TEST3=y_test3
y_TEST3[y_TEST3 ==" No Heart Disease"]=0
y_TEST3[y_TEST3 =="Heart Disease"] = 1
print(rocauc(y_TEST3, rfc_prob3[:,1]))

0.7152116402116402


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """


The resutls of all three models are really interesting as we can indicate that the random forest model with more variables should have more overall accuracy and better AUC scores as well. However, the model 2 with 2 variables and model 3 with 3 variables have really close model accuracy with OA at 0.67 and 0.66. And the 3 variable model doesn't really assure a better result. Our model 1 result is a bit unrealistic since it is literally score close to 1, therefore this could lead to inaccurate result. But I believe the overall accuracy still reflect to the max featrue numbers since more variables can provide better result.  